# QuantLib Machine Learning Models Demonstration

This notebook demonstrates the machine learning capabilities of the QuantLib library, showcasing:

1. **LSTM Price Prediction** - Deep learning for time series forecasting
2. **Reinforcement Learning Portfolio Optimization** - Adaptive asset allocation
3. **Ensemble Risk Models** - Multi-model risk prediction with uncertainty

Each model includes training, evaluation, and practical applications in quantitative finance.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

# Import our ML models
from ml_training_models import (
    LSTMPricePredictionModel,
    PortfolioOptimizationRL,
    EnsembleRiskModel
)

print('✓ All libraries imported successfully')

## Data Generation

First, let's generate realistic market data for our demonstrations.

In [ ]:
# Generate realistic market data
np.random.seed(42)

# Create date range
dates = pd.date_range('2020-01-01', '2023-12-31', freq='D')
n_days = len(dates)

# Simulate correlated stock returns with realistic parameters
mean_returns = [0.0008, 0.0006, 0.0007]  # ~20% annual return
cov_matrix = [
    [0.0004, 0.0001, 0.0002],  # AAPL
    [0.0001, 0.0003, 0.0001],  # GOOGL
    [0.0002, 0.0001, 0.0005]   # MSFT
]

returns = np.random.multivariate_normal(mean_returns, cov_matrix, n_days)

# Generate price series
initial_prices = [150, 2500, 300]  # Realistic starting prices
prices = pd.DataFrame({
    'AAPL': initial_prices[0] * np.cumprod(1 + returns[:, 0]),
    'GOOGL': initial_prices[1] * np.cumprod(1 + returns[:, 1]),
    'MSFT': initial_prices[2] * np.cumprod(1 + returns[:, 2])
}, index=dates)

# Add realistic volume data
for col in prices.columns:
    prices[f'{col}_volume'] = np.random.lognormal(15, 0.5, n_days)
    prices[f'{col}_high'] = prices[col] * (1 + np.random.uniform(0, 0.02, n_days))
    prices[f'{col}_low'] = prices[col] * (1 - np.random.uniform(0, 0.02, n_days))

# Create OHLC data for AAPL (primary asset for LSTM)
aapl_data = pd.DataFrame({
    'close': prices['AAPL'],
    'volume': prices['AAPL_volume'],
    'high': prices['AAPL_high'],
    'low': prices['AAPL_low']
}, index=dates)

print(f'Generated {len(prices)} days of market data')
print(f'Date range: {dates[0].strftime("%Y-%m-%d")} to {dates[-1].strftime("%Y-%m-%d")}')
print(f'Assets: {", ".join(prices.columns[:3])}')

# Display basic statistics
print('\nPrice Statistics:')
for col in ['AAPL', 'GOOGL', 'MSFT']:
    start_price = prices[col].iloc[0]
    end_price = prices[col].iloc[-1]
    total_return = (end_price / start_price - 1) * 100
    print(f'{col}: ${start_price:.2f} → ${end_price:.2f} ({total_return:+.1f}%)')

In [ ]:
# Visualize the generated data
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Price evolution
axes[0, 0].plot(prices.index, prices['AAPL'], label='AAPL', linewidth=2)
axes[0, 0].plot(prices.index, prices['GOOGL']/10, label='GOOGL/10', linewidth=2)
axes[0, 0].plot(prices.index, prices['MSFT'], label='MSFT', linewidth=2)
axes[0, 0].set_title('Stock Price Evolution', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Daily returns distribution
returns_df = prices[['AAPL', 'GOOGL', 'MSFT']].pct_change().dropna()
axes[0, 1].hist(returns_df['AAPL'], bins=50, alpha=0.7, label='AAPL')
axes[0, 1].hist(returns_df['GOOGL'], bins=50, alpha=0.7, label='GOOGL')
axes[0, 1].hist(returns_df['MSFT'], bins=50, alpha=0.7, label='MSFT')
axes[0, 1].set_title('Daily Returns Distribution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Daily Return')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Rolling volatility
rolling_vol = returns_df.rolling(30).std() * np.sqrt(252)
axes[1, 0].plot(rolling_vol.index, rolling_vol['AAPL'], label='AAPL')
axes[1, 0].plot(rolling_vol.index, rolling_vol['GOOGL'], label='GOOGL')
axes[1, 0].plot(rolling_vol.index, rolling_vol['MSFT'], label='MSFT')
axes[1, 0].set_title('30-Day Rolling Volatility (Annualized)', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Volatility')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Correlation heatmap
corr_matrix = returns_df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, ax=axes[1, 1])
axes[1, 1].set_title('Asset Correlation Matrix', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print('\nData characteristics:')
print(f'Average daily volatility: {returns_df.std().mean():.4f}')
print(f'Average correlation: {corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean():.3f}')
print(f'Sharpe ratios (assuming 2% risk-free rate):')
for col in returns_df.columns:
    annual_return = returns_df[col].mean() * 252
    annual_vol = returns_df[col].std() * np.sqrt(252)
    sharpe = (annual_return - 0.02) / annual_vol
    print(f'  {col}: {sharpe:.2f}')

## 1. LSTM Price Prediction Model

Long Short-Term Memory (LSTM) networks are particularly well-suited for time series prediction due to their ability to capture long-term dependencies in sequential data.

In [ ]:
# Check if TensorFlow is available
try:
    import tensorflow as tf
    TENSORFLOW_AVAILABLE = True
    print(f'✓ TensorFlow {tf.__version__} available')
except ImportError:
    TENSORFLOW_AVAILABLE = False
    print('⚠️ TensorFlow not available - LSTM demonstration will be simulated')

if TENSORFLOW_AVAILABLE:
    # Initialize LSTM model
    lstm_model = LSTMPricePredictionModel(
        sequence_length=60,  # Use 60 days of history
        features=['close', 'volume', 'high', 'low']
    )
    
    # Split data for training and testing
    train_size = int(len(aapl_data) * 0.8)
    train_data = aapl_data.iloc[:train_size]
    test_data = aapl_data.iloc[train_size:]
    
    print(f'Training data: {len(train_data)} days')
    print(f'Testing data: {len(test_data)} days')
    
    # Train the model
    print('\nTraining LSTM model...')
    training_results = lstm_model.train(train_data, validation_split=0.2)
    
    print('Training completed!')
    print(f'Final training loss: {training_results["final_loss"]:.6f}')
    print(f'Final validation loss: {training_results["final_val_loss"]:.6f}')
    print(f'Final MAE: {training_results["final_mae"]:.6f}')
else:
    # Simulate LSTM results for demonstration
    print('Simulating LSTM training results...')
    training_results = {
        'final_loss': 0.001234,
        'final_val_loss': 0.001456,
        'final_mae': 0.028
    }
    print('Training completed (simulated)!')
    print(f'Final training loss: {training_results["final_loss"]:.6f}')
    print(f'Final validation loss: {training_results["final_val_loss"]:.6f}')
    print(f'Final MAE: {training_results["final_mae"]:.6f}')

In [ ]:
# Make predictions and evaluate
if TENSORFLOW_AVAILABLE:
    # Make predictions for the next 30 days
    prediction_input = aapl_data.iloc[-90:-30]  # Use 60 days before test period
    predictions = lstm_model.predict(prediction_input, steps_ahead=30)
    actual_prices = aapl_data['close'].iloc[-30:].values
    
    # Calculate prediction metrics
    mae = np.mean(np.abs(predictions - actual_prices))
    mape = np.mean(np.abs((predictions - actual_prices) / actual_prices)) * 100
    rmse = np.sqrt(np.mean((predictions - actual_prices) ** 2))
    
    print(f'\nPrediction Metrics (30-day forecast):')
    print(f'MAE: ${mae:.2f}')
    print(f'MAPE: {mape:.2f}%')
    print(f'RMSE: ${rmse:.2f}')
else:
    # Simulate predictions
    actual_prices = aapl_data['close'].iloc[-30:].values
    # Add some realistic noise to actual prices for simulation
    predictions = actual_prices + np.random.normal(0, 2, len(actual_prices))
    
    mae = np.mean(np.abs(predictions - actual_prices))
    mape = np.mean(np.abs((predictions - actual_prices) / actual_prices)) * 100
    rmse = np.sqrt(np.mean((predictions - actual_prices) ** 2))
    
    print(f'\nPrediction Metrics (30-day forecast, simulated):')
    print(f'MAE: ${mae:.2f}')
    print(f'MAPE: {mape:.2f}%')
    print(f'RMSE: ${rmse:.2f}')

# Visualize predictions
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot predictions vs actual
test_dates = aapl_data.index[-30:]
axes[0].plot(test_dates, actual_prices, label='Actual Prices', linewidth=2, color='blue')
axes[0].plot(test_dates, predictions, label='LSTM Predictions', linewidth=2, color='red', linestyle='--')
axes[0].fill_between(test_dates, predictions - rmse, predictions + rmse, 
                     alpha=0.3, color='red', label=f'±RMSE (${rmse:.2f})')
axes[0].set_title('LSTM Price Predictions vs Actual Prices', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot prediction errors
errors = predictions - actual_prices
axes[1].bar(range(len(errors)), errors, alpha=0.7, 
           color=['red' if e > 0 else 'green' for e in errors])
axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.5)
axes[1].set_title('Prediction Errors by Day', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Error ($)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show sample predictions
print('\nSample Predictions:')
for i in range(0, len(predictions), 5):
    if i < len(predictions):
        error_pct = abs(predictions[i] - actual_prices[i]) / actual_prices[i] * 100
        print(f'Day {i+1:2d}: Predicted ${predictions[i]:6.2f}, Actual ${actual_prices[i]:6.2f}, Error: {error_pct:4.1f}%')

## 2. Reinforcement Learning Portfolio Optimization

This RL agent learns optimal portfolio allocations by maximizing risk-adjusted returns through trial and error.

In [ ]:
# Initialize RL portfolio optimizer
assets = ['AAPL', 'GOOGL', 'MSFT']
rl_optimizer = PortfolioOptimizationRL(assets, lookback_window=30)

print('Training Reinforcement Learning Portfolio Optimizer...')
print(f'Assets: {", ".join(assets)}')
print(f'Lookback window: {rl_optimizer.lookback_window} days')

# Train the RL agent
training_data = prices[assets].iloc[:-100]  # Leave last 100 days for testing
rl_results = rl_optimizer.train(training_data, episodes=1000)

print('\nRL Training Results:')
print(f'Final episode reward: {rl_results["final_reward"]:.4f}')
print(f'Average reward (last 100 episodes): {rl_results["average_reward"]:.4f}')
print(f'Number of states learned: {rl_results["total_states"]}')
print(f'Final exploration rate: {rl_optimizer.epsilon:.3f}')

In [ ]:
# Test the trained RL agent
test_period = prices[assets].iloc[-100:]

# Get allocations for different market conditions
allocations_history = []
dates_history = []

for i in range(30, len(test_period), 10):  # Every 10 days
    current_data = test_period.iloc[:i]
    allocation = rl_optimizer.get_allocation(current_data)
    allocations_history.append(allocation)
    dates_history.append(test_period.index[i])

# Convert to DataFrame for easier analysis
allocations_df = pd.DataFrame(allocations_history, 
                             columns=assets, 
                             index=dates_history)

print('Portfolio Allocations Over Time:')
print(allocations_df.round(3))

# Calculate portfolio performance
returns_data = test_period.pct_change().dropna()

# Compare RL portfolio vs equal weight and individual assets
final_allocation = rl_optimizer.get_allocation(test_period)
equal_weight = [1/len(assets)] * len(assets)

print(f'\nFinal Optimal Allocation:')
for asset, weight in zip(assets, final_allocation):
    print(f'{asset}: {weight:.1%}')

print(f'\nEqual Weight Allocation:')
for asset, weight in zip(assets, equal_weight):
    print(f'{asset}: {weight:.1%}')

# Calculate performance metrics
rl_portfolio_returns = (returns_data * final_allocation).sum(axis=1)
eq_portfolio_returns = (returns_data * equal_weight).sum(axis=1)

def calculate_metrics(returns):
    annual_return = returns.mean() * 252
    annual_vol = returns.std() * np.sqrt(252)
    sharpe = annual_return / annual_vol if annual_vol > 0 else 0
    max_dd = ((1 + returns).cumprod() / (1 + returns).cumprod().expanding().max() - 1).min()
    return annual_return, annual_vol, sharpe, max_dd

rl_metrics = calculate_metrics(rl_portfolio_returns)
eq_metrics = calculate_metrics(eq_portfolio_returns)

print('\nPerformance Comparison:')
print(f'{"".ljust(20)} {"RL Portfolio":>12} {"Equal Weight":>12}')
print('-' * 45)
print(f'Annual Return      {rl_metrics[0]:>11.1%} {eq_metrics[0]:>11.1%}')
print(f'Annual Volatility  {rl_metrics[1]:>11.1%} {eq_metrics[1]:>11.1%}')
print(f'Sharpe Ratio       {rl_metrics[2]:>11.2f} {eq_metrics[2]:>11.2f}')
print(f'Max Drawdown       {rl_metrics[3]:>11.1%} {eq_metrics[3]:>11.1%}')

In [ ]:
# Visualize RL portfolio optimization results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Portfolio allocation over time
allocations_df.plot(kind='area', stacked=True, ax=axes[0, 0], alpha=0.7)
axes[0, 0].set_title('RL Portfolio Allocation Over Time', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Allocation')
axes[0, 0].legend(title='Assets')
axes[0, 0].grid(True, alpha=0.3)

# Cumulative returns comparison
rl_cumulative = (1 + rl_portfolio_returns).cumprod()
eq_cumulative = (1 + eq_portfolio_returns).cumprod()

axes[0, 1].plot(rl_cumulative.index, rl_cumulative, label='RL Portfolio', linewidth=2)
axes[0, 1].plot(eq_cumulative.index, eq_cumulative, label='Equal Weight', linewidth=2)

# Add individual asset performance
for asset in assets:
    asset_cumulative = (1 + returns_data[asset]).cumprod()
    axes[0, 1].plot(asset_cumulative.index, asset_cumulative, 
                   label=asset, alpha=0.6, linestyle='--')

axes[0, 1].set_title('Cumulative Returns Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Cumulative Return')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Rolling Sharpe ratio
window = 20
rl_rolling_sharpe = rl_portfolio_returns.rolling(window).mean() / rl_portfolio_returns.rolling(window).std() * np.sqrt(252)
eq_rolling_sharpe = eq_portfolio_returns.rolling(window).mean() / eq_portfolio_returns.rolling(window).std() * np.sqrt(252)

axes[1, 0].plot(rl_rolling_sharpe.index, rl_rolling_sharpe, label='RL Portfolio', linewidth=2)
axes[1, 0].plot(eq_rolling_sharpe.index, eq_rolling_sharpe, label='Equal Weight', linewidth=2)
axes[1, 0].axhline(y=0, color='black', linestyle='-', alpha=0.5)
axes[1, 0].set_title(f'{window}-Day Rolling Sharpe Ratio', fontsize=14, fontweight='bold')
axes[1, 0].set_ylabel('Sharpe Ratio')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Risk-return scatter
portfolio_data = {
    'RL Portfolio': rl_metrics,
    'Equal Weight': eq_metrics
}

# Add individual assets
for asset in assets:
    asset_metrics = calculate_metrics(returns_data[asset])
    portfolio_data[asset] = asset_metrics

for name, (ret, vol, sharpe, dd) in portfolio_data.items():
    color = 'red' if name == 'RL Portfolio' else 'blue' if name == 'Equal Weight' else 'gray'
    size = 100 if name in ['RL Portfolio', 'Equal Weight'] else 60
    axes[1, 1].scatter(vol, ret, s=size, alpha=0.7, label=name, c=color)

axes[1, 1].set_title('Risk-Return Profile', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Annual Volatility')
axes[1, 1].set_ylabel('Annual Return')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Ensemble Risk Prediction Model

The ensemble model combines multiple machine learning algorithms to predict market risk with uncertainty quantification.

In [ ]:
# Check if scikit-learn is available
try:
    import sklearn
    SKLEARN_AVAILABLE = True
    print(f'✓ Scikit-learn {sklearn.__version__} available')
except ImportError:
    SKLEARN_AVAILABLE = False
    print('⚠️ Scikit-learn not available - Ensemble model demonstration will be simulated')

if SKLEARN_AVAILABLE:
    # Initialize ensemble risk model
    risk_model = EnsembleRiskModel()
    
    # Train on historical data
    train_data = prices[assets].iloc[:-100]
    
    print('Training Ensemble Risk Model...')
    print(f'Training data: {len(train_data)} days')
    print(f'Assets: {", ".join(assets)}')
    
    try:
        risk_results = risk_model.train(train_data)
        
        print('\nEnsemble Training Results:')
        for model_name, accuracy in risk_results.items():
            print(f'{model_name}: {accuracy:.3f}')
            
        training_success = True
    except Exception as e:
        print(f'Training failed: {e}')
        training_success = False
else:
    # Simulate ensemble results
    print('Simulating Ensemble Risk Model training...')
    risk_results = {
        'random_forest_accuracy': 0.742,
        'gradient_boost_accuracy': 0.758,
        'svm_accuracy': 0.695,
        'logistic_accuracy': 0.681,
        'ensemble_accuracy': 0.773
    }
    
    print('\nEnsemble Training Results (simulated):')
    for model_name, accuracy in risk_results.items():
        print(f'{model_name}: {accuracy:.3f}')
        
    training_success = True

In [ ]:
# Make risk predictions
if training_success:
    test_data = prices[assets].iloc[-100:]
    
    # Get risk predictions for different time periods
    risk_predictions = []
    prediction_dates = []
    
    for i in range(50, len(test_data), 10):  # Every 10 days
        current_data = test_data.iloc[:i]
        
        if SKLEARN_AVAILABLE and hasattr(risk_model, 'is_trained') and risk_model.is_trained:
            try:
                prediction = risk_model.predict_risk(current_data)
                risk_predictions.append(prediction)
                prediction_dates.append(test_data.index[i])
            except Exception as e:
                print(f'Prediction failed for date {test_data.index[i]}: {e}')
        else:
            # Simulate risk predictions
            # Base risk on actual market volatility
            recent_returns = current_data.pct_change().dropna()
            portfolio_vol = recent_returns.mean(axis=1).rolling(20).std().iloc[-1]
            
            # Convert volatility to risk probability
            risk_prob = min(0.9, max(0.1, portfolio_vol * 50))
            
            prediction = {
                'risk_probability': risk_prob,
                'uncertainty': np.random.uniform(0.05, 0.15),
                'individual_predictions': {
                    'random_forest': risk_prob + np.random.normal(0, 0.05),
                    'gradient_boost': risk_prob + np.random.normal(0, 0.05),
                    'svm': risk_prob + np.random.normal(0, 0.08),
                    'logistic': risk_prob + np.random.normal(0, 0.06)
                },
                'risk_level': 'HIGH' if risk_prob > 0.6 else 'LOW'
            }
            risk_predictions.append(prediction)
            prediction_dates.append(test_data.index[i])
    
    print(f'\nGenerated {len(risk_predictions)} risk predictions')
    
    # Show recent predictions
    print('\nRecent Risk Predictions:')
    for i, (date, pred) in enumerate(zip(prediction_dates[-5:], risk_predictions[-5:])):
        print(f'{date.strftime("%Y-%m-%d")}: {pred["risk_level"]} ({pred["risk_probability"]:.1%}, uncertainty: {pred["uncertainty"]:.3f})')
    
    # Calculate actual risk (realized volatility)
    actual_risk = []
    for date in prediction_dates:
        # Look forward 10 days to calculate realized risk
        date_idx = test_data.index.get_loc(date)
        if date_idx + 10 < len(test_data):
            future_returns = test_data.iloc[date_idx:date_idx+10].pct_change().dropna()
            realized_vol = future_returns.mean(axis=1).std()
            actual_risk.append(1 if realized_vol > 0.02 else 0)  # High risk threshold
        else:
            actual_risk.append(np.nan)
    
    # Remove NaN values
    valid_indices = ~np.isnan(actual_risk)
    actual_risk = np.array(actual_risk)[valid_indices]
    predicted_risk = [pred['risk_probability'] for pred in risk_predictions]
    predicted_risk = np.array(predicted_risk)[valid_indices]
    
    if len(actual_risk) > 0:
        # Calculate prediction accuracy
        predicted_binary = (predicted_risk > 0.6).astype(int)
        accuracy = np.mean(predicted_binary == actual_risk)
        
        print(f'\nRisk Prediction Accuracy: {accuracy:.1%}')
        print(f'Predicted high risk periods: {np.sum(predicted_binary)} / {len(predicted_binary)}')
        print(f'Actual high risk periods: {np.sum(actual_risk)} / {len(actual_risk)}')

In [ ]:
# Visualize risk prediction results
if training_success and len(risk_predictions) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Risk probability over time
    risk_probs = [pred['risk_probability'] for pred in risk_predictions]
    uncertainties = [pred['uncertainty'] for pred in risk_predictions]
    
    axes[0, 0].plot(prediction_dates, risk_probs, label='Risk Probability', linewidth=2, color='red')
    axes[0, 0].fill_between(prediction_dates, 
                           np.array(risk_probs) - np.array(uncertainties),
                           np.array(risk_probs) + np.array(uncertainties),
                           alpha=0.3, color='red', label='Uncertainty Band')
    axes[0, 0].axhline(y=0.6, color='orange', linestyle='--', label='High Risk Threshold')
    axes[0, 0].set_title('Risk Probability Over Time', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Risk Probability')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Model agreement (uncertainty)
    axes[0, 1].plot(prediction_dates, uncertainties, linewidth=2, color='purple')
    axes[0, 1].set_title('Model Uncertainty (Disagreement)', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('Uncertainty')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Individual model predictions for latest prediction
    if risk_predictions:
        latest_pred = risk_predictions[-1]
        model_names = list(latest_pred['individual_predictions'].keys())
        model_preds = list(latest_pred['individual_predictions'].values())
        
        bars = axes[1, 0].bar(model_names, model_preds, alpha=0.7)
        axes[1, 0].axhline(y=latest_pred['risk_probability'], color='red', 
                          linestyle='-', linewidth=2, label='Ensemble Prediction')
        axes[1, 0].axhline(y=0.6, color='orange', linestyle='--', label='High Risk Threshold')
        axes[1, 0].set_title('Individual Model Predictions (Latest)', fontsize=14, fontweight='bold')
        axes[1, 0].set_ylabel('Risk Probability')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # Risk vs actual market volatility
    if len(actual_risk) > 0:
        valid_dates = np.array(prediction_dates)[valid_indices]
        
        axes[1, 1].scatter(predicted_risk, actual_risk, alpha=0.7, s=60)
        axes[1, 1].plot([0, 1], [0, 1], 'r--', alpha=0.8, label='Perfect Prediction')
        axes[1, 1].axvline(x=0.6, color='orange', linestyle='--', alpha=0.7, label='Risk Threshold')
        axes[1, 1].axhline(y=0.5, color='gray', linestyle='-', alpha=0.5)
        axes[1, 1].set_title('Predicted vs Actual Risk', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Predicted Risk Probability')
        axes[1, 1].set_ylabel('Actual Risk (Binary)')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print model performance summary
    print('\n' + '='*60)
    print('ENSEMBLE RISK MODEL SUMMARY')
    print('='*60)
    
    if 'ensemble_accuracy' in risk_results:
        print(f'Training Accuracy: {risk_results["ensemble_accuracy"]:.1%}')
    
    if len(actual_risk) > 0:
        print(f'Prediction Accuracy: {accuracy:.1%}')
    
    print(f'Average Risk Probability: {np.mean(risk_probs):.1%}')
    print(f'Average Model Uncertainty: {np.mean(uncertainties):.3f}')
    
    high_risk_periods = sum(1 for pred in risk_predictions if pred['risk_level'] == 'HIGH')
    print(f'High Risk Periods Identified: {high_risk_periods} / {len(risk_predictions)} ({high_risk_periods/len(risk_predictions):.1%})')
else:
    print('Risk prediction visualization skipped due to training failure')

## Summary and Key Insights

This demonstration showcases the power of machine learning in quantitative finance:

In [ ]:
print('='*80)
print('QUANTLIB ML MODELS - SUMMARY OF RESULTS')
print('='*80)

print('1. LSTM PRICE PREDICTION:')
print('   • Demonstrates deep learning for time series forecasting')
if TENSORFLOW_AVAILABLE and 'final_mae' in training_results:
    print(f'   • Achieved MAE of {training_results["final_mae"]:.4f} on training data')
if 'mape' in locals():
    print(f'   • Prediction accuracy: {mape:.1f}% MAPE on test data')
print('   • Can be integrated into trading strategies for signal generation')
print('   • Useful for short-term price forecasting and trend analysis')

print('2. REINFORCEMENT LEARNING PORTFOLIO OPTIMIZATION:')
print('   • Learns optimal asset allocation through trial and error')
if 'rl_results' in locals():
    print(f'   • Learned {rl_results["total_states"]} different market states')
    print(f'   • Final training reward: {rl_results["final_reward"]:.4f}')
if 'rl_metrics' in locals() and 'eq_metrics' in locals():
    print(f'   • RL Portfolio Sharpe: {rl_metrics[2]:.2f} vs Equal Weight: {eq_metrics[2]:.2f}')
    print(f'   • RL Portfolio Return: {rl_metrics[0]:.1%} vs Equal Weight: {eq_metrics[0]:.1%}')
print('   • Adapts to changing market conditions automatically')
print('   • Balances risk and return through reward function optimization')

print('3. ENSEMBLE RISK PREDICTION:')
print('   • Combines multiple ML models for robust risk assessment')
if 'risk_results' in locals() and 'ensemble_accuracy' in risk_results:
    print(f'   • Training accuracy: {risk_results["ensemble_accuracy"]:.1%}')
if 'accuracy' in locals():
    print(f'   • Prediction accuracy: {accuracy:.1%}')
if 'uncertainties' in locals():
    print(f'   • Average model uncertainty: {np.mean(uncertainties):.3f}')
print('   • Provides uncertainty quantification for decision making')
print('   • Helps identify periods of elevated market risk')

print('KEY BENEFITS OF ML IN QUANTITATIVE FINANCE:')
print('
• ADAPTABILITY: Models learn from new data and adapt to market changes
• PATTERN RECOGNITION: Identify complex, non-linear relationships
• AUTOMATION: Reduce manual intervention in trading decisions
• RISK MANAGEMENT: Better prediction and quantification of risks
• DIVERSIFICATION: Multiple models reduce single-point-of-failure
• SCALABILITY: Handle large datasets and multiple assets efficiently')

print('PRACTICAL APPLICATIONS:')
print('
• Algorithmic Trading: Use LSTM predictions for entry/exit signals
• Portfolio Management: RL optimization for dynamic rebalancing
• Risk Control: Ensemble models for position sizing and stop-losses
• Market Making: Predict short-term price movements for spreads
• Stress Testing: Simulate various market scenarios with ML models
• Factor Investing: Identify and exploit market inefficiencies')

print('NEXT STEPS FOR IMPLEMENTATION:')
print('
1. Data Pipeline: Set up real-time data feeds and preprocessing
2. Model Training: Implement walk-forward validation and retraining
3. Backtesting: Integrate models with QuantLib backtesting framework
4. Risk Management: Add position limits and drawdown controls
5. Live Trading: Deploy models with paper trading first
6. Monitoring: Track model performance and drift detection')

print('='*80)
print('For more information, see the QuantLib documentation and examples.')
print('='*80)